In [1]:
!pip3 install ultralytics
!pip3 install opendatasets
!pip3 install pytube opencv-python
!pip3 install ffmpeg

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 677.8/677.8 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 1.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for ffmpeg: filename=ffmpeg-1.4-py3-none-any.whl size=6080 sha256=961e7302cad61906bab02af6652a2865691e797531d6ac6baa4704a0e45f9f79
  Stored in directory: /root/.cache/pip/wheels/8e/7a/69/cd6aeb83b126a7f04cbe7c9d929028dc52a6e7d525ff56003a
Successfully built ffmpeg


In [15]:
import pandas as pd
import os
import shutil
from sklearn.model_selection import train_test_split
from IPython import display
from ultralytics import YOLO
import opendatasets as od
import xml.etree.ElementTree as ET
from pytube import YouTube
import cv2
import requests
import subprocess
from glob import glob
import random
import yaml

## Download training images

In [3]:
od.download("https://www.kaggle.com/datasets/lyly99/logodet3k")
# you must have a kaggle account and generate your api in the settings

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: leaaumagy
Your Kaggle Key: ··········


100%|██████████| 2.87G/2.87G [00:39<00:00, 78.1MB/s]


## Copy the 'Clothes' folder to a new 'Dataset' folder

In [4]:
def copy_and_rename(folder_path, destination_path):
    # Copy the "Clothes" folder to the new "Dataset" folder
    shutil.copytree(folder_path, os.path.join(destination_path))

# Path of the 'Clothes' folder
folder_path = '/content/logodet3k/LogoDet-3K/Clothes'

# Path of the destination folder
destination_path = '/content/Dataset/Clothes'

# Call the function to copy
copy_and_rename(folder_path, destination_path)

## Rename brand folder names

In [5]:
def rename_folders(directory_path):
    # List all items in the directory
    items = os.listdir(directory_path)

    for item in items:
        item_path = os.path.join(directory_path, item)

        # Check if the item is a folder
        if os.path.isdir(item_path):
            # Rename the folder by replacing spaces with underscores
            new_name = item.replace(" ", "_")
            new_name = new_name.replace("76", "A_76")
            new_name = new_name.replace("-", "_")
            new_name = new_name.replace("'", "_")
            new_path = os.path.join(directory_path, new_name)
            os.rename(item_path, new_path)

# Directory to be processed
directory_to_process = '/content/Dataset/Clothes'
rename_folders(directory_to_process)

## Filter on selected brands

In [6]:
# Path to the "Clothes" folder
clothes_path = "/content/Dataset/Clothes"

# Get the list of all files and folders in the "Clothes" folder
all_items = os.listdir(clothes_path)

# Display all file and folder names
print("List of all files and folders in 'Clothes':")
print(all_items)

List of all files and folders in 'Clothes':
['tiffany', 'stussy', 'loro_piana', 'Adidas_SB', 'Celine', 'Man_O__War', 'adyson', 'Ariat', 'Capezio', 'AOKANG', 'Armani_Exchange', 'Altra', 'American_Apparel', 'Acne_Studios', 'Herschel_Supply', 'sansabelt', 'norrona', 'Anne_Klein', 'Andrew_Marc', 'pajar', 'oris', 'vivobarefoot_2', 'new_balance_1', 'new_balance_2', 'miu_miu', 'Clarks', 'Bulgari', 'Boscov_s', 'youngor_2', 'superga_2', 'Covert', 'mido', 'topman', 'Carhartt', 'Grundies', 'EKCO', 'Everlast', 'gigo', 'brace_yourself', 'rocky', 'Audemars_Piguet', 'wolford', '7_PE', 'Imperial', 'Josef_Seibel', 'Adriatica', 'marchesa', 'maui_jim', 'Guy_Laroche', 'Kiton', 'Chippewa', 'moncler_2', 'Hoka_One_One', 'wrangler', 'majestic', 'zara', 'rapha', 'Admiral', 'Galvin_Green', 'u.s._polo_assn', 'swatch', 'Kenneth_Cole', 'Guess', 'A._Favre_&_Fils', 'sanuk', 'Andrew_Christian', 'Bulova', 'pythia', 'corneliani', 'chaps', 'zingara', 'Boxfresh', 'triumph_1', 'Chipotle_Mexican_Grill_2', 'youngor_3', 'ser

In [7]:
# Path to the "Clothes" folder
clothes_path = "/content/Dataset/Clothes"

# List of selected folders
selected_folders = ['Adidas_SB', 'American_Eagle', 'Armani', 'Balenciaga',
                    'Bulgari', 'Ck_Calvin_Klein_1',
                    'Ck_Calvin_Klein_2', 'Canada Goose', 'Carhartt', 'Casio', 'Celine',
                    'Champion', 'chanel', 'Chloe', 'Columbia', 'Converse',
                    'Ellesse', 'Everlast', 'Gap', 'Giorgio', 'Guess',
                    'Hugo_Boss', 'Karl_Lagerfield', 'Kenzo', 'lacoste', 'levis',
                    'louis_vuitton_1', 'louis_vuitton_2', 'napapijri',
                    'new_balance_1', 'new_balance_2', 'obey', 'oxxford', 'patagonia', 'pepe_jeans',
                    'polo_ralph_lauren', 'prada', 'rolex', 'sergio_tacchini',
                    'the_timberland', 'the_north_face', 'timberland', 'tommy_hilfiger', 'tom_ford',
                    'uniqlo', 'versace', 'zara']

# Get the list of all files and folders in the "Clothes" folder
all_items = os.listdir(clothes_path)

# Filter folders using the selected list
filtered_folders = [folder for folder in all_items if os.path.isdir(os.path.join(clothes_path, folder)) and folder.lower() in [sf.lower() for sf in selected_folders]]

# Display filtered folders
print("Filtered folders:")
print(filtered_folders)

# Iterate through all folders in the "Clothes" folder
for folder in all_items:
    folder_path = os.path.join(clothes_path, folder)

    # Check if the folder is not in the list of selected folders
    if folder.lower() not in [sf.lower() for sf in selected_folders] and os.path.isdir(folder_path):
        # Remove the non-selected folder and its contents
        shutil.rmtree(folder_path)

print("Operation completed.")

Filtered folders:
['Adidas_SB', 'Celine', 'new_balance_1', 'new_balance_2', 'Bulgari', 'Carhartt', 'Everlast', 'zara', 'Guess', 'sergio_tacchini', 'louis_vuitton_1', 'lacoste', 'prada', 'Champion', 'Kenzo', 'levis', 'Gap', 'uniqlo', 'Ck_Calvin_Klein_1', 'oxxford', 'the_timberland', 'Giorgio', 'American_Eagle', 'Balenciaga', 'patagonia', 'Ck_Calvin_Klein_2', 'pepe_jeans', 'Casio', 'Columbia', 'Converse', 'versace', 'rolex', 'tom_ford', 'obey', 'tommy_hilfiger', 'Hugo_Boss', 'timberland', 'polo_ralph_lauren', 'Armani', 'louis_vuitton_2', 'the_north_face', 'Ellesse', 'chanel', 'Chloe', 'napapijri']
Operation completed.


## Rename images and xml file names to avoid duplicates

In [8]:
def rename_files(folder_path):
    for root, dirs, files in os.walk(folder_path):
        # Exclude non-numeric subdirectories
        dirs = [d for d in dirs if d.isdigit()]

        for file in files:
            if file.endswith(".jpg") or file.endswith(".xml"):
                # Get the parent folder number
                parent_folder_name = os.path.basename(root)

                # Build the new file name with the parent folder number
                new_name = f"{os.path.splitext(file)[0]}_{parent_folder_name}{os.path.splitext(file)[1]}"

                # Current file path
                current_path = os.path.join(root, file)

                # Build the new file path
                new_path = os.path.join(root, new_name)

                # Rename the file
                os.rename(current_path, new_path)

# Path of the 'Clothes' folder
folder_path = '/content/Dataset/Clothes'

# Call the function to rename files
rename_files(folder_path)

## Balancing the number of instances between brands

In [9]:
# Main directory path
main_directory = '/content/Dataset/Clothes'

# Function to count files with a specific extension in a directory
def count_files(directory, extension):
    # Use glob to find all files with the specified extension
    files = glob(os.path.join(directory, f'*.{extension.lower()}'))
    number_of_files = len(files)
    return number_of_files

# Iterate through all folders in the main directory
for subdirectory in os.listdir(main_directory):
    subdirectory_path = os.path.join(main_directory, subdirectory)

    # Check if it's a directory
    if os.path.isdir(subdirectory_path):
        # Count JPEG files in this directory
        number_of_jpeg = count_files(subdirectory_path, 'jpeg') + count_files(subdirectory_path, 'jpg')

        # Count XML files in this directory
        number_of_xml = count_files(subdirectory_path, 'xml')

        # Display the result
        print(f"The folder {subdirectory} contains {number_of_jpeg} JPEG files and {number_of_xml} XML files.")

The folder Adidas_SB contains 65 JPEG files and 65 XML files.
The folder Celine contains 108 JPEG files and 108 XML files.
The folder new_balance_1 contains 155 JPEG files and 155 XML files.
The folder new_balance_2 contains 155 JPEG files and 155 XML files.
The folder Bulgari contains 80 JPEG files and 80 XML files.
The folder Carhartt contains 31 JPEG files and 31 XML files.
The folder Everlast contains 55 JPEG files and 55 XML files.
The folder zara contains 151 JPEG files and 151 XML files.
The folder Guess contains 50 JPEG files and 50 XML files.
The folder sergio_tacchini contains 22 JPEG files and 22 XML files.
The folder louis_vuitton_1 contains 96 JPEG files and 96 XML files.
The folder lacoste contains 55 JPEG files and 55 XML files.
The folder prada contains 52 JPEG files and 52 XML files.
The folder Champion contains 29 JPEG files and 29 XML files.
The folder Kenzo contains 63 JPEG files and 63 XML files.
The folder levis contains 76 JPEG files and 76 XML files.
The folder 

In [12]:
def count_files(directory, extension):
    files = glob(os.path.join(directory, f'*.{extension}'))
    number_of_files = len(files)
    return number_of_files

def duplicate_files(folder_path, target_count_per_folder):
    for subfolder in os.listdir(folder_path):
        subfolder_path = os.path.join(folder_path, subfolder)

        if os.path.isdir(subfolder_path):
            # Calculate the number of images to duplicate to reach at least target_count_per_folder
            images_to_duplicate = max(0, target_count_per_folder - count_files(subfolder_path, 'jpg'))

            # Duplicate files if necessary
            for i in range(images_to_duplicate):
                # New names for duplicated files
                new_image_number = images_to_duplicate + i + 1
                new_image_name = f"{new_image_number}_{subfolder}.jpg"
                new_xml_name = f"{new_image_number}_{subfolder}.xml"
                new_image_path = os.path.join(subfolder_path, new_image_name)
                new_xml_path = os.path.join(subfolder_path, new_xml_name)

                # Use a random image and XML file for duplication
                source_image_path = random.choice(glob(os.path.join(subfolder_path, '*.jpg')))
                source_xml_path = source_image_path.replace('.jpg', '.xml')

                # Check if the source file is different from the destination file
                if source_image_path != new_image_path:
                    shutil.copy(source_image_path, new_image_path)
                if source_xml_path != new_xml_path:
                    shutil.copy(source_xml_path, new_xml_path)

folder_path = "/content/Dataset/Clothes"
target_count_per_folder = 150
duplicate_files(folder_path, target_count_per_folder)

## Creation of the Dataset (train, test, valid) and the YALM file

In [13]:
import os
import shutil
from sklearn.model_selection import train_test_split

# Path to the main folder
main_folder = '/content/Dataset/Clothes'

# Create the folder structure for the Dataset
dataset_folder = '/content/Dataset'
folders = ['train', 'test', 'valid']
subfolders = ['images', 'labels']

# Create the folder structure
for folder in folders:
    for subfolder in subfolders:
        path = os.path.join(dataset_folder, folder, subfolder)
        os.makedirs(path, exist_ok=True)

# Get the list of all brand folders in your Clothes folder
brand_folders = [f for f in os.listdir(main_folder) if os.path.isdir(os.path.join(main_folder, f))]
num_classes = len(brand_folders)

# Initialize variables outside the loop
all_train_files = []
all_test_files = []
all_valid_files = []

# Split data into train, test, and validation sets for each brand
for brand_folder in brand_folders:
    brand_path = os.path.join(main_folder, brand_folder)
    image_files = [f for f in os.listdir(brand_path) if f.endswith('.jpg')]

    # Ensure there are enough samples for the split
    if len(image_files) < 5:
        print(f"Assigning all samples from {brand_folder} to the training set due to insufficient samples.")
        train_files = image_files
        test_files = []
        valid_files = []
    else:
        # Split into train and test_valid_files
        train_files, test_valid_files = train_test_split(image_files, test_size=0.2, random_state=43)

        # Check if test_valid_files has enough samples
        if len(test_valid_files) < 2:
            train_files.extend(test_valid_files)
            test_files = []
            valid_files = []
        else:
            # Split test_valid_files into test and validation
            test_files, valid_files = train_test_split(test_valid_files, test_size=0.5, random_state=43)

    # Add files from each brand to the overall list
    all_train_files.extend(train_files)
    all_test_files.extend(test_files)
    all_valid_files.extend(valid_files)

    # Move files to the corresponding folders
    def move_files(source_folder, destination_folder, file_list):
        for file in file_list:
            image_path = os.path.join(brand_path, file)
            xml_path = os.path.join(brand_path, os.path.splitext(file)[0] + '.xml')

            dest_image_path = os.path.join(dataset_folder, destination_folder, 'images', file)
            dest_xml_path = os.path.join(dataset_folder, destination_folder, 'labels', os.path.splitext(file)[0] + '.xml')

            shutil.copy(image_path, dest_image_path)
            shutil.copy(xml_path, dest_xml_path)

    # Move files to the corresponding folders
    move_files(brand_folder, 'train', train_files)
    move_files(brand_folder, 'test', test_files)
    move_files(brand_folder, 'valid', valid_files)

# Create a YAML file for YOLO
yolo_yaml_content = "names:\n"

for idx, brand_name in enumerate(brand_folders):
    yolo_yaml_content += f"  {idx}: {brand_name}\n"

yolo_yaml_content += f"""
train: {os.path.join(dataset_folder, 'train/images')}
val: {os.path.join(dataset_folder, 'valid/images')}
test: {os.path.join(dataset_folder, 'test/images')}

nc: {num_classes}
"""

with open('/content/Dataset/data.yaml', 'w') as yaml_file:
    yaml_file.write(yolo_yaml_content)

## Converting XML files to TXT

In [16]:
# Load the YAML file with names and associated IDs
with open('/content/Dataset/data.yaml', 'r') as yaml_file:
    yaml_content = yaml.load(yaml_file, Loader=yaml.FullLoader)

class_ids = {v: k for k, v in yaml_content['names'].items()}

def convert_xml_to_yolov8(xml_path, output_path):
    tree = ET.parse(xml_path)
    root = tree.getroot()

    image_width = int(root.find(".//size/width").text)
    image_height = int(root.find(".//size/height").text)

    with open(output_path, 'w') as output_file:
        for obj in root.findall(".//object"):
            raw_class_name = obj.find("name").text
            # Replace spaces with underscores
            class_name = raw_class_name.replace(" ", "_")
            class_name = class_name.replace("-", "_")
            class_name = class_name.replace("'", "_")
            class_name = class_name.replace("76", "A_76")
            class_name = class_name.replace("snow_peak_2", "timbuk2_1")
            class_name = class_name.replace("Alcatel_2", "alpinestars_2")
            class_name = class_name.replace("maybach_1", "meyba_1")
            class_name = class_name.replace("INOHERB_2", "Isabel_Maran_2")
            class_name = class_name.replace("Harvest_Moon", "Heat_1")
            class_name = class_name.replace("Alcatel_2", "alpinestars_2")
            class_name = class_name.replace("Alcatel_2", "alpinestars_2")
            class_name = class_name.replace("Alcatel_2", "alpinestars_2")
            class_name = class_name.replace("Alcatel_2", "alpinestars_2")
            class_name = class_name.replace("Alcatel_2", "alpinestars_2")
            class_name = class_name.replace("Alcatel_2", "alpinestars_2")
            class_name = class_name.replace("Alcatel_2", "alpinestars_2")

            # Use the get method with a default value of -1
            class_id = class_ids.get(class_name, -1)

            # Check if the class was found
            if class_id == -1:
                print(f"Class not found for {class_name} in the file {xml_path}")
                continue

            xmin = int(obj.find("bndbox/xmin").text)
            ymin = int(obj.find("bndbox/ymin").text)
            xmax = int(obj.find("bndbox/xmax").text)
            ymax = int(obj.find("bndbox/ymax").text)

            center_x = (xmin + xmax) / (2.0 * image_width)
            center_y = (ymin + ymax) / (2.0 * image_height)
            bbox_width = (xmax - xmin) / image_width
            bbox_height = (ymax - ymin) / image_height

            output_line = f"{class_id} {center_x} {center_y} {bbox_width} {bbox_height}\n"
            output_file.write(output_line)

def process_xml_folder(xml_folder):
    yolov8_output_folder = xml_folder  # Same output folder as the input folder

    # Ensure the output folder exists
    os.makedirs(yolov8_output_folder, exist_ok=True)

    # Conversion for each XML file
    for xml_file in os.listdir(xml_folder):
        if xml_file.endswith(".xml"):
            xml_path = os.path.join(xml_folder, xml_file)
            yolov8_output_path = os.path.join(yolov8_output_folder, xml_file.replace(".xml", ".txt"))
            convert_xml_to_yolov8(xml_path, yolov8_output_path)

            # Remove the XML file after conversion
            os.remove(xml_path)

# Folders to process
input_folders = [
    '/content/Dataset/train/labels',
    '/content/Dataset/valid/labels',
    '/content/Dataset/test/labels'
]

# Process each XML folder
for xml_folder in input_folders:
    process_xml_folder(xml_folder)

## Loading the YOLO version 8 model

In [17]:
# Clear the output in Jupyter Notebook
display.clear_output()

# Your YOLO command
!yolo checks

Ultralytics YOLOv8.0.235 🚀 Python-3.10.12 torch-2.1.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 30.4/78.2 GB disk)

OS                  Linux-6.1.58+-x86_64-with-glibc2.35
Environment         Colab
Python              3.10.12
Install             pip
RAM                 12.67 GB
CPU                 Intel Xeon 2.20GHz
CUDA                12.1

matplotlib          ✅ 3.7.1>=3.3.0
numpy               ✅ 1.23.5>=1.22.2
opencv-python       ✅ 4.8.0.76>=4.6.0
pillow              ✅ 9.4.0>=7.1.2
pyyaml              ✅ 6.0.1>=5.3.1
requests            ✅ 2.31.0>=2.23.0
scipy               ✅ 1.11.4>=1.4.1
torch               ✅ 2.1.0+cu121>=1.8.0
torchvision         ✅ 0.16.0+cu121>=0.9.0
tqdm                ✅ 4.66.1>=4.64.0
psutil              ✅ 5.9.5
py-cpuinfo          ✅ 9.0.0
thop                ✅ 0.1.1-2209072238>=0.1.1
pandas              ✅ 1.5.3>=1.1.4
seaborn             ✅ 0.12.2>=0.11.0


In [18]:
# Load a model
model = YOLO("yolov8m.yaml")  # build a new model from scratch
model = YOLO("yolov8m.pt")  # load a pretrained model (recommended for training)


                   from  n    params  module                                       arguments                     
  0                  -1  1      1392  ultralytics.nn.modules.conv.Conv             [3, 48, 3, 2]                 
  1                  -1  1     41664  ultralytics.nn.modules.conv.Conv             [48, 96, 3, 2]                
  2                  -1  2    111360  ultralytics.nn.modules.block.C2f             [96, 96, 2, True]             
  3                  -1  1    166272  ultralytics.nn.modules.conv.Conv             [96, 192, 3, 2]               
  4                  -1  4    813312  ultralytics.nn.modules.block.C2f             [192, 192, 4, True]           
  5                  -1  1    664320  ultralytics.nn.modules.conv.Conv             [192, 384, 3, 2]              
  6                  -1  4   3248640  ultralytics.nn.modules.block.C2f             [384, 384, 4, True]           
  7                  -1  1   1991808  ultralytics.nn.modules.conv.Conv             [384

100%|██████████| 49.7M/49.7M [00:00<00:00, 218MB/s]


In [19]:
!yolo task=detect mode=train model=/content/yolov8m.pt data=/content/Dataset/data.yaml epochs = 30 imgsz = 640

Ultralytics YOLOv8.0.235 🚀 Python-3.10.12 torch-2.1.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=/content/yolov8m.pt, data=/content/Dataset/data.yaml, epochs=30, time=None, patience=50, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=Tr

In [20]:
!yolo task=detect mode=val model=/content/runs/detect/train/weights/best.pt data=/content/Dataset/data.yaml

Ultralytics YOLOv8.0.235 🚀 Python-3.10.12 torch-2.1.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 218 layers, 25865815 parameters, 0 gradients, 78.8 GFLOPs
val: Scanning /content/Dataset/valid/labels.cache... 648 images, 0 backgrounds, 0 corrupt: 100% 648/648 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% 41/41 [00:14<00:00,  2.74it/s]
                   all        648        801      0.972      0.954      0.977        0.9
             Adidas_SB        648         17      0.995      0.882      0.907       0.78
                Celine        648         12      0.972      0.917      0.989      0.808
         new_balance_1        648         32      0.993          1      0.995      0.941
         new_balance_2        648         21      0.988          1      0.995      0.924
               Bulgari        648         13      0.986          1      0.995      0.908
              Carhartt        648         15      

## Logo detection on a list of sample videos

In [21]:
# Function to convert processed videos to mp4 format
def convert_avi_to_mp4(input_video_path, output_video_path):
    input_video = cv2.VideoCapture(input_video_path)
    width = int(input_video.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(input_video.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = input_video.get(cv2.CAP_PROP_FPS)
    fourcc = cv2.VideoWriter_fourcc(*"mp4v")
    output_video = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))

    while True:
        ret, frame = input_video.read()
        if not ret:
            break
        output_video.write(frame)

    input_video.release()
    output_video.release()

# Function to detect folders starting with 'predict' where processed videos are saved
def process_detect_folders(detect_path):
    for folder_name in os.listdir(detect_path):
        folder_path = os.path.join(detect_path, folder_name)

        if os.path.isdir(folder_path) and folder_name.startswith("predict"):
            for file_name in os.listdir(folder_path):
                file_path = os.path.join(folder_path, file_name)

                if os.path.isfile(file_path) and file_name.lower().endswith(".avi"):
                    # Create an output path with the same filename but with the .mp4 extension
                    mp4_path = os.path.join(folder_path, file_name.lower().replace(".avi", ".mp4"))

                    # Convert AVI video to MP4
                    convert_avi_to_mp4(file_path, mp4_path)

                    # Remove the AVI file after conversion
                    os.remove(file_path)

# List of your videos
video_urls = [
    "https://youtu.be/XNF1dlHlOd0?si=0dOK_xMNDRHXJqIB",
    "https://youtube.com/shorts/-U3vC3swKAo?si=VI2kdgJhBmdhmZWm",
    "https://youtube.com/shorts/tWPu7SHJOKM?si=cNQpuhSkfqX_ehwj"
]

# Folder where original videos are saved
save_path = '/content/video_youtube'

# Folder to save videos generated by YOLO
yolo_save_path = '/content/runs/detect/predict'

# Specify the path to the /content/runs/detect folder where folders starting with 'predict' are located
detect_folder_path = "/content/runs/detect"

# Extract video IDs from URLs
video_ids = [url.split("/")[-1].split("?")[0] for url in video_urls]

# Create a DataFrame
df = pd.DataFrame(index=selected_folders, columns=video_ids)

# Loop for each video
for video_url in video_urls:
    # Download the video
    yt = YouTube(video_url)
    video_stream = yt.streams.get_highest_resolution()
    video_path = video_stream.download(save_path)

    # Video ID associated with the video
    video_id = video_url.split("/")[-1].split("?")[0]

    # Rename the video file with the ID
    new_video_name = f'{video_id}.mp4'
    new_video_path = os.path.join(save_path, new_video_name)
    os.rename(video_path, new_video_path)

    # YOLO command
    yolo_command = f"yolo task=detect mode=predict model=/content/runs/detect/train/weights/best.pt source={new_video_path}"
    result_yolo = subprocess.run(yolo_command, shell=True, capture_output=True, text=True)

    # Call the function that looks for and converts the processed video to mp4 format
    process_detect_folders(detect_folder_path)

    # Analyze the content of result_yolo.stdout for each element of each row in the DataFrame
    for folder in selected_folders:
        if folder in result_yolo.stdout:
            df.loc[folder, video_id] = 'Yes'
        else:
            df.loc[folder, video_id] = 'No'

# Display the final DataFrame with results
print(df)

                  XNF1dlHlOd0 -U3vC3swKAo tWPu7SHJOKM
Adidas_SB                 Yes         Yes          No
American_Eagle             No          No          No
Armani                     No          No          No
Balenciaga                 No         Yes          No
Bulgari                   Yes          No          No
Ck_Calvin_Klein_1          No          No          No
Ck_Calvin_Klein_2          No          No          No
Canada Goose               No          No          No
Carhartt                   No          No          No
Casio                      No          No          No
Celine                     No         Yes          No
Champion                   No          No          No
chanel                     No          No          No
Chloe                      No          No          No
Columbia                  Yes          No          No
Converse                   No          No          No
Ellesse                   Yes         Yes         Yes
Everlast                   N